In [1]:
import pandas as pd
import os

In [ ]:
WORK_DIR=''

In [2]:
df = pd.read_csv(os.path.join(WORK_DIR, 'data/causal_text/tirzepatide/filtered_clean.csv'))
df_original = pd.read_csv(os.path.join(WORK_DIR, 'natural/data/tirzepatide/filter_posts_tirzepatide_gpt-3.5-turbo_relevance_filtered_posts_relevant.csv'))

In [3]:
import ast
all_outcome_words = []
for sublist in df_original['outcome_words']:
    sublist = ast.literal_eval(sublist)
    all_outcome_words.extend(sublist)
all_outcome_words = list(set(all_outcome_words))

all_treatment_words = []
for sublist in df_original['treatments']:
    sublist = ast.literal_eval(sublist)
    all_treatment_words.extend(sublist)
all_treatment_words = list(set(all_treatment_words))
print(all_outcome_words)
print(all_treatment_words)

['glucose', 'hb', 'pound', 'kilo', 'weigh', 'kg', 'gain', 'drop', 'lost', 'hemoglobin', 'a1c', 'lb', 'sugar', 'loss']
['rybelsu', 'semaglutide', 'tirzepatid', 'ozempec', 'rybelsus', 'semaglutid', 'semaglitude', 'ozempic', 'wegovy', 'ozmepic', 'mounjero', 'mounjao', 'mounjaro', 'wegov', 'wegoby', 'ribelsus', 'mounjoro', 'tirzapetide', 'rybelus', 'ryblesus', 'ozepmic', 'semaglutde', 'munjaro', 'ozempci', 'tirzepatide', 'rybelsis']


In [4]:
df.head()

,age,sex,country,bmi,start_HbA1c,start_weight,duration_days,target_achieved,drug_type,post
0,45.0,1.0,0.0,28.5,7.5,190.0,21.0,1,1,\r\n\r\n## Subreddit \r\n> This comment was fo...
1,42.0,1.0,0.0,28.5,7.8,220.0,7.0,0,1,\r\n\r\n## Subreddit \r\n> This comment was fo...
2,46.0,1.0,0.0,28.5,7.2,200.0,42.0,1,1,\r\n\r\n## Subreddit \r\n> This comment was fo...
3,45.0,1.0,0.0,27.5,7.8,200.0,56.0,1,1,\r\n\r\n## Subreddit \r\n> This comment was fo...
4,45.0,1.0,0.0,27.5,7.2,200.0,30.0,1,1,\r\n\r\n## Subreddit \r\n> This comment was fo...


In [5]:
df['post'] = df['post'].str.strip()

In [6]:
import re

all_mask_words = all_treatment_words + all_outcome_words
# Escape special characters and create regex pattern for words
word_pattern = r'\b(?:' + '|'.join(map(re.escape, all_mask_words)) + r')\b'

# Create a regex pattern to match numbers including those adjacent to letters
number_pattern = r'\d+'

# Combine both patterns (words and numbers)
combined_pattern = f'({word_pattern})|({number_pattern})'

# Replace all occurrences with [MASK]
df['post_masked'] = df['post'].str.replace(combined_pattern, '[MASK]', regex=True, flags=re.IGNORECASE)

In [7]:
df['post_masked'].values

array(['## Subreddit \r\n> This comment was found on the subreddit r/[MASK].\r\n\r\n## Title \r\n> This comment was in response to a post titled: r/[MASK] Lounge\r\n\r\n## Date Created \r\n> This comment was created on November [MASK], [MASK].\r\n\r\n## Comment \r\n> I have [MASK] [MASK] lbs now and just took my [MASK]rd shot this AM. I am blown away.\r\n\r\n## Output \r\n>',
       '## Subreddit \r\n> This comment was found on the subreddit r/[MASK].\r\n\r\n## Title \r\n> This comment was in response to a post titled: r/[MASK] Lounge\r\n\r\n## Date Created \r\n> This comment was created on November [MASK], [MASK].\r\n\r\n## Comment \r\n> First week on MJ down [MASK].[MASK] lbs! Is that normal?? I feel like it’s crazy compared to others I know it will slow down but jeez\r\n\r\n## Output \r\n>',
       '## Subreddit \r\n> This comment was found on the subreddit r/[MASK].\r\n\r\n## Title \r\n> This comment was in response to a post titled: r/[MASK] Lounge\r\n\r\n## Date Created \r\n> Thi

In [8]:
# Step 1: Remove everything before and including "## Comment \r\n>"
df['comment_masked'] = df['post_masked'].str.replace(r'(?s).*?## Comment\s*\r\n>', '', regex=True)

# Step 2: Remove "## Output \r\n>" and everything after it
df['comment_masked'] = df['comment_masked'].str.replace(r'## Output\s*\r\n>.*', '', regex=True)

# Step 3: Strip leading/trailing whitespace and newline characters
df['comment_masked'] = df['comment_masked'].str.strip()

In [9]:
df['comment_masked'].values

array(['I have [MASK] [MASK] lbs now and just took my [MASK]rd shot this AM. I am blown away.',
       'First week on MJ down [MASK].[MASK] lbs! Is that normal?? I feel like it’s crazy compared to others I know it will slow down but jeez',
       'I didn’t notice much of a change till I started [MASK] mg-the first month I was unsure I wanted to stay on it, but when I started [MASK], it was BAM, this stuff is a miracle! 😂 I’m taking my [MASK]th dose of [MASK] mg tomorrow morning-it’s still working pretty good-I’ve [MASK] [MASK] lbs this last month',
       ...,
       'Anyone here taking a weight [MASK] medication called [MASK]? \r\n\r\nI was [MASK]lbs last year and now I hover [MASK]-[MASK]lbs depending on water weight. The thing is I love myself but I want to breath again and not have painful joints. This drug has helped me with [MASK]lbs so far since late August. I\'m just struggling with how long it\'s taking and the fear of failing. Overall, I don\'t ever want to be skinny, just a 

In [10]:
# Step 1: Remove everything before and including "## Comment \r\n>"
df['comment'] = df['post'].str.replace(r'(?s).*?## Comment\s*\r\n>', '', regex=True)

# Step 2: Remove "## Output \r\n>" and everything after it
df['comment'] = df['comment'].str.replace(r'## Output\s*\r\n>.*', '', regex=True)

# Step 3: Strip leading/trailing whitespace and newline characters
df['comment'] = df['comment'].str.strip()

In [11]:
df.head()

,age,sex,country,bmi,start_HbA1c,start_weight,duration_days,target_achieved,drug_type,post,post_masked,comment_masked,comment
0,45.0,1.0,0.0,28.5,7.5,190.0,21.0,1,1,## Subreddit \r\n> This comment was found on t...,## Subreddit \r\n> This comment was found on t...,I have [MASK] [MASK] lbs now and just took my ...,I have lost 10 lbs now and just took my 3rd sh...
1,42.0,1.0,0.0,28.5,7.8,220.0,7.0,0,1,## Subreddit \r\n> This comment was found on t...,## Subreddit \r\n> This comment was found on t...,First week on MJ down [MASK].[MASK] lbs! Is th...,First week on MJ down 7.5 lbs! Is that normal?...
2,46.0,1.0,0.0,28.5,7.2,200.0,42.0,1,1,## Subreddit \r\n> This comment was found on t...,## Subreddit \r\n> This comment was found on t...,I didn’t notice much of a change till I starte...,I didn’t notice much of a change till I starte...
3,45.0,1.0,0.0,27.5,7.8,200.0,56.0,1,1,## Subreddit \r\n> This comment was found on t...,## Subreddit \r\n> This comment was found on t...,Yes. The same happened to me on [MASK].[MASK] ...,Yes. The same happened to me on 2.5 and I reme...
4,45.0,1.0,0.0,27.5,7.2,200.0,30.0,1,1,## Subreddit \r\n> This comment was found on t...,## Subreddit \r\n> This comment was found on t...,I didn’t monitor it but I [MASK] [MASK]lbs tha...,I didn’t monitor it but I lost 10lbs that mont...


In [12]:
df.to_csv(os.path.join(WORK_DIR, 'data/causal_text/tirzepatide/filtered_clean_masked.csv'), index=False)